In [ ]:
from somefunctions import *
augmented = False

# Configuration parameters
Easier if we put them all togheter

In [ ]:
reg = 1e-6  #regularization term
pool_size = (2, 2)
train_shape = 400 #size of the training images
patch_size = 16
input_size = 80 # for now is the patch size
pad_size = int(input_size/2 - patch_size/2)
batch_size = 100
learning_rate = 0.001


nb_epoch = 2 #very small, only preliminary tests
steps_per_epoch = 50 #the number of training samples is huge, arbitrary value
        

# Function to generate the minibatch
This function will be called during the training. <br>
Here we can also do some feature augmentation, for example. But maybe is more efficient to flip all the images before. <br>
TODO: take a random patch of dimension (input_size x input_size)

In [ ]:
def generate_minibatch(X,Y):
    """
    Generate a minibatch
    """
    while 1:
        # Generate one minibatch
        X_batch = np.empty((batch_size, input_size, input_size, 3))
        Y_batch = np.empty(batch_size)
        low=input_size//2
        high = (train_shape + 2*pad_size - input_size//2)
        for i in range(batch_size):
            # Select a random image
            idx = np.random.choice(X.shape[0])
            
            x_coord = np.random.randint(low=low, high = high ) 
            y_coord = np.random.randint(low=low, high = high )
      
            X_batch[i] = X[idx,x_coord - input_size//2:x_coord + input_size//2,y_coord - input_size//2:y_coord + input_size//2]
            
            gt_batch = Y[idx,x_coord - patch_size//2:x_coord + patch_size//2,y_coord - patch_size//2:y_coord + patch_size//2]            
            Y_batch[i] = patch_to_label(gt_batch)
            
            #TODO: TRY TO CHANGE CONTRAST AND BRIGHTNESS
            
            
        yield X_batch, Y_batch

# Train
Generate the model and train the dataset

In [ ]:
def create_model():
    model = Sequential()
    model.add(Convolution2D(64, kernel_size=(5, 5), 
                            input_shape = ( input_size, input_size, 3), 
                            activation = 'relu',
                            padding = 'SAME'))
    model.add(Flatten())
    model.add(Dense(units = 128, activation = 'relu'))
    model.add(Dense(units = 1, activation = 'sigmoid'))

    opt = Adam(lr=learning_rate) # Adam optimizer with default initial learning rate
 

    # This callback reduces the learning rate when the training accuracy does not improve any more
    lr_callback = ReduceLROnPlateau(monitor='acc', factor=0.5, patience=5,
                                    verbose=1, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)
    
    # Stops the training process upon convergence
    stop_callback = EarlyStopping(monitor='acc', min_delta=0.0001, patience=11, verbose=1, mode='auto')
    
    model.compile(loss=binary_crossentropy,
                  optimizer=opt,
                  metrics=['acc'])
    
    return model, stop_callback, lr_callback

    

def train(X, Y):    
    '''
    Generate an instance of the model an train the model on X, Y
    '''
    print('Training set shape: ', X.shape) 
    print(f'Batch_size: {batch_size} \nSteps per epoch: {steps_per_epoch} \n')
    
    
    model, stop_callback, lr_callback = create_model()
    
    np.random.seed(20122018) # Reproducibility + remember the deadline is the 20.12.2018
    
    try:
        model.fit_generator(generate_minibatch(X,Y),
                            steps_per_epoch=steps_per_epoch,
                            nb_epoch=nb_epoch,
                            verbose=1,
                            callbacks=[lr_callback, stop_callback])
    except KeyboardInterrupt:
        print('\nKeyboard interruption!')
        pass
    
    print('Training completed')
    
    model.save_weights('pesi')
    
    return model

# Load images

In [ ]:
# Load a set of image
root_dir = "../Data/training/"

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n = len(files) 
print("Loading " + str(n) + " images")
imgs_original = [load_image(image_dir + files[i]) for i in range(n)]
imgs = [load_image(image_dir + files[i]) for i in range(n)]
print(files[0])

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " images")
gt_imgs = [load_image(gt_dir + files[i]) for i in range(n)]
print(files[0])

# Preprocessing

In [ ]:
#augmenting
if not augmented:
    #flip
    for i in range(n):
        imgs.append(np.fliplr(imgs[i]))
        imgs.append(np.flipud(imgs[i]))
        gt_imgs.append(np.fliplr(gt_imgs[i]))
        gt_imgs.append(np.flipud(gt_imgs[i]))
    
    #rotate
    for i in range(3*n):
        imgs.append(np.rot90(imgs[i],1))
        imgs.append(np.rot90(imgs[i],2))
        imgs.append(np.rot90(imgs[i],3))
        gt_imgs.append(np.rot90(gt_imgs[i],1))
        gt_imgs.append(np.rot90(gt_imgs[i],2))
        gt_imgs.append(np.rot90(gt_imgs[i],3))
        
    augmented = True
        

imgs = padding_imgs(np.array(imgs),pad_size)
gt_imgs = padding_GT(np.array(gt_imgs),pad_size)

# Training

# split in validation + train

(VERY BASIC!) In this way we can check directly what happens in the validation set

In [ ]:
N = imgs.shape[0]
ratio = 0.8
N_train = int(N*ratio)
N_valid = int(N - N*ratio)
X_train = imgs[:N_train]
X_valid = imgs[N-N_valid:]
Y_train = gt_imgs[:N_train]
Y_valid = gt_imgs[N-N_valid:]

In [ ]:
print(X_train.shape)
print(X_valid.shape)
print(X_train.shape[0] + X_valid.shape[0])

In [ ]:
model = train(X_train,Y_train)

# Check if on the train set we have good results

In [ ]:
def imgs_to_windows(imgs, img_size, patch_size, window_size):
    ''' Takes padded images and outputs an array with the windows'''
    windows = []
    for idx in range(imgs.shape[0]):
        im = imgs[idx]
        for i in range(img_size//patch_size):
            for j in range(img_size//patch_size):
                temp = im[j*patch_size:window_size + j*patch_size,
                                  i*patch_size:window_size + i*patch_size]
                windows.append(temp)
    return np.asarray(windows)


In [ ]:
def label_to_img(imgwidth, imgheight, w, h, labels):
    im = np.zeros([imgwidth, imgheight])
    idx = 0
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            im[j:j+w, i:i+h] = labels[idx]
            idx = idx + 1
    return im

In [ ]:
IDX = 1
X = imgs_to_windows(imgs[1:3],400,patch_size,input_size)
patches_idx = X[625*IDX:625*(IDX+1)]

patches_idx.shape


In [ ]:
Z_idx = model.predict(patches_idx)
labels_idx = (Z_idx > 0.5) *1 
predicted_image = label_to_img(400,400,16,16,labels_idx)
    

In [ ]:
plt.imshow(concatenate_images(imgs_original[IDX],predicted_image))

# predict on test

In [ ]:
test_images = np.asarray(pick_test_images())
test_images = padding_imgs(np.array(test_images),pad_size)


n_test = test_images.shape[0]

test_inputs = imgs_to_windows(test_images,608,16,80)
print(test_inputs.shape)

model, _,_ = create_model()
model.load_weights('pesi')

In [ ]:
Z = model.predict(test_inputs)

In [ ]:
labels1D = (Z < 0.5 ) * 1
labels1D.shape
labels = labels1D.reshape(50,-1)
print(labels.shape)

In [ ]:
IDX = 9
im = label_to_img(608, 608, 16, 16, labels[IDX])
plt.imshow(concatenate_images(test_images[IDX], im))

In [ ]:
MY_masks_to_submission('dummsdsy_submission.csv', labels)